In [40]:
import sys
sys.path.append("../")

import pandas as pd
import plotly.graph_objects as go
from dateutil import parser
import datetime as dt

from api.oanda_api import OandaApi
from infrastructure.instrument_collection import instrumentCollection as ic
from db.db import DataDB

pd.set_option('display.max_rows', None)

In [41]:
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [2]:
database = DataDB()

In [3]:
ic.LoadInstrumentsDB()

In [4]:
ic.instruments_dict

{'USD_HKD': {'name': 'USD_HKD', 'ins_type': 'CURRENCY', 'displayName': 'USD/HKD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 5},
 'CHF_ZAR': {'name': 'CHF_ZAR', 'ins_type': 'CURRENCY', 'displayName': 'CHF/ZAR', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 5},
 'EUR_ZAR': {'name': 'EUR_ZAR', 'ins_type': 'CURRENCY', 'displayName': 'EUR/ZAR', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.05, 'displayPrecision': 5},
 'AUD_JPY': {'name': 'AUD_JPY', 'ins_type': 'CURRENCY', 'displayName': 'AUD/JPY', 'pipLocation': 0.01, 'tradeUnitsPrecision': 0, 'marginRate': 0.0333, 'displayPrecision': 3},
 'XAG_EUR': {'name': 'XAG_EUR', 'ins_type': 'METAL', 'displayName': 'Silver/EUR', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.1, 'displayPrecision': 5},
 'AUD_USD': {'name': 'AUD_USD', 'ins_type': 'CURRENCY', 'displayName': 'AUD/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision'

In [5]:
pairs = []
for pair, val in ic.instruments_dict.items():
    if val.ins_type == 'CURRENCY' and "GBP" in pair:
        pairs.append(pair)

In [6]:
pairs

['GBP_ZAR',
 'GBP_HKD',
 'GBP_CHF',
 'GBP_AUD',
 'GBP_CAD',
 'GBP_SGD',
 'GBP_USD',
 'GBP_PLN',
 'EUR_GBP',
 'GBP_NZD',
 'GBP_JPY']

In [7]:
data = []
api = OandaApi()
for p in pairs:
    print(p)
    d_temp = api.get_candles_df(pair_name = p, granularity="D", count=100)
    d_temp['gain'] = ((d_temp.mid_c - d_temp.mid_o) / d_temp.mid_c) * 100
    d_temp['pair'] = p
    if '_GBP' in p:
        d_temp['gain'] = d_temp['gain'] * -1
    data.append(d_temp)
candles_df = pd.concat(data)
candles_df.reset_index(drop=True, inplace=True)
candles_df.time = candles_df.time.dt.date

GBP_ZAR
GBP_HKD
GBP_CHF
GBP_AUD
GBP_CAD
GBP_SGD
GBP_USD
GBP_PLN
EUR_GBP
GBP_NZD
GBP_JPY


In [8]:
candles_df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
0,2024-08-27,81142,23.50200,23.59858,23.45468,23.51225,23.4535,23.58961,23.43450,23.4795,23.5505,23.60765,23.46100,23.54500,0.043594,GBP_ZAR
1,2024-08-28,82376,23.51600,23.53884,23.29209,23.40561,23.4590,23.52700,23.27637,23.3695,23.5730,23.58150,23.30781,23.44172,-0.471639,GBP_ZAR
2,2024-08-29,96013,23.38725,23.44311,23.20490,23.42100,23.3330,23.43381,23.19700,23.3850,23.4415,23.47420,23.21280,23.45700,0.144101,GBP_ZAR
3,2024-09-01,48588,23.42475,23.55436,23.38080,23.43275,23.3855,23.54531,23.36150,23.3875,23.4640,23.56372,23.38874,23.47800,0.034140,GBP_ZAR
4,2024-09-02,85618,23.43300,23.60156,23.41113,23.57725,23.3900,23.59500,23.37450,23.5620,23.4760,23.62172,23.41850,23.59250,0.611819,GBP_ZAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,2025-01-08,246668,195.75300,195.82000,193.48100,194.59000,195.6200,195.80400,193.46600,194.5490,195.8860,195.88600,193.49600,194.63200,-0.597667,GBP_JPY
1085,2025-01-09,329125,194.61900,194.77900,192.20700,192.56600,194.4820,194.76400,192.19300,192.4830,194.7560,194.81000,192.22000,192.65000,-1.066128,GBP_JPY
1086,2025-01-12,314286,192.59800,192.76100,190.08000,192.17800,192.3730,192.74700,190.06300,192.1110,192.8230,192.82300,190.09600,192.24600,-0.218547,GBP_JPY
1087,2025-01-13,326263,192.20000,193.01400,191.49000,192.97600,192.0460,192.98500,191.47300,192.9420,192.3530,193.05300,191.50800,193.00900,0.402123,GBP_JPY


In [9]:
calendar_data = database.query_all(DataDB.CALENDAR_COLL)

In [10]:
calendar_data_df = pd.DataFrame.from_dict(calendar_data)

In [11]:
calendar_data_df.date = calendar_data_df.date.dt.date

In [12]:
calendar_data_df_uk = calendar_data_df[calendar_data_df.country == 'united kingdom'].copy()

In [16]:
calendar_data_df_uk

,date,country,category,event,symbol,actual,previous,forecast
22,2024-12-13,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.10,0.00,0.20
27,2024-12-17,united kingdom,unemployment rate,unemployment rate,UKUEILOR,4.30,0.00,4.30
33,2024-12-18,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.60,0.00,2.63
40,2024-12-19,united kingdom,interest rate,boe interest rate decision,UKBRBASE,4.75,0.00,4.75
49,2024-12-13,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.10,0.00,0.20
...,...,...,...,...,...,...,...,...
1010,2024-12-19,united kingdom,interest rate,boe interest rate decision,UKBRBASE,4.75,4.75,4.75
1013,2024-12-20,united kingdom,retail sales mom,retail sales mom,GBRRetailSalesMoM,0.20,-0.70,0.50
1048,2025-01-15,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.50,2.60,2.70
1055,2025-01-16,united kingdom,monthly gdp mom,gdp mom,GBRMGM,0.00,-0.10,0.10


In [17]:
calendar_data_df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93 entries, 22 to 1056
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      93 non-null     object 
 1   country   93 non-null     object 
 2   category  93 non-null     object 
 3   event     93 non-null     object 
 4   symbol    93 non-null     object 
 5   actual    93 non-null     float64
 6   previous  93 non-null     float64
 7   forecast  93 non-null     float64
dtypes: float64(3), object(5)
memory usage: 6.5+ KB


In [15]:
for col in ['actual', 'previous', 'forecast']:
    for sy in ['%', 'B', 'K', '£']:
        calendar_data_df_uk[col] = calendar_data_df_uk[col].str.replace(sy, "")

    calendar_data_df_uk[col].replace('', 0, inplace=True)
    calendar_data_df_uk[col] = calendar_data_df_uk[col].astype(float)

/var/folders/20/fx1vp06158bgqhgm1r3vsjhw0000gn/T/ipykernel_8124/1787204613.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  calendar_data_df_uk[col].replace('', 0, inplace=True)
/var/folders/20/fx1vp06158bgqhgm1r3vsjhw0000gn/T/ipykernel_8124/1787204613.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [18]:
calendar_data_df_uk.info()

<class 'pandas.core.frame.DataFrame'>
Index: 93 entries, 22 to 1056
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      93 non-null     object 
 1   country   93 non-null     object 
 2   category  93 non-null     object 
 3   event     93 non-null     object 
 4   symbol    93 non-null     object 
 5   actual    93 non-null     float64
 6   previous  93 non-null     float64
 7   forecast  93 non-null     float64
dtypes: float64(3), object(5)
memory usage: 6.5+ KB


In [24]:
calendar_data_df_uk

,date,country,category,event,symbol,actual,previous,forecast,orig_date
22,2024-12-15,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.10,0.00,0.20,2024-12-13
27,2024-12-17,united kingdom,unemployment rate,unemployment rate,UKUEILOR,4.30,0.00,4.30,2024-12-17
33,2024-12-18,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.60,0.00,2.63,2024-12-18
40,2024-12-19,united kingdom,interest rate,boe interest rate decision,UKBRBASE,4.75,0.00,4.75,2024-12-19
49,2024-12-15,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.10,0.00,0.20,2024-12-13
...,...,...,...,...,...,...,...,...,...
1010,2024-12-19,united kingdom,interest rate,boe interest rate decision,UKBRBASE,4.75,4.75,4.75,2024-12-19
1013,2024-12-22,united kingdom,retail sales mom,retail sales mom,GBRRetailSalesMoM,0.20,-0.70,0.50,2024-12-20
1048,2025-01-21,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.50,2.60,2.70,2025-01-15
1055,2025-01-22,united kingdom,monthly gdp mom,gdp mom,GBRMGM,0.00,-0.10,0.10,2025-01-16


In [22]:
date_match = {}
for orig in calendar_data_df_uk.date.unique():
    d = orig
    tries = 0
    while d not in candles_df.time.values:
        d = d + dt.timedelta(days = 1)
        tries += 1
        if tries > 5:
            break
    
    date_match[orig] = d
    
            

Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed
Failed


In [23]:
calendar_data_df_uk['orig_date'] = calendar_data_df_uk.date
calendar_data_df_uk.date = [date_match[x] for x in calendar_data_df_uk.date]

In [25]:
calendar_data_df_uk['delta_prev'] = calendar_data_df_uk.actual - calendar_data_df_uk.previous
calendar_data_df_uk['delta_fc'] = calendar_data_df_uk.actual - calendar_data_df_uk.forecast

In [26]:
calendar_data_df_uk

,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
22,2024-12-15,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.10,0.00,0.20,2024-12-13,-0.10,-0.30
27,2024-12-17,united kingdom,unemployment rate,unemployment rate,UKUEILOR,4.30,0.00,4.30,2024-12-17,4.30,0.00
33,2024-12-18,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.60,0.00,2.63,2024-12-18,2.60,-0.03
40,2024-12-19,united kingdom,interest rate,boe interest rate decision,UKBRBASE,4.75,0.00,4.75,2024-12-19,4.75,0.00
49,2024-12-15,united kingdom,monthly gdp mom,gdp mom,GBRMGM,-0.10,0.00,0.20,2024-12-13,-0.10,-0.30
...,...,...,...,...,...,...,...,...,...,...,...
1010,2024-12-19,united kingdom,interest rate,boe interest rate decision,UKBRBASE,4.75,4.75,4.75,2024-12-19,0.00,0.00
1013,2024-12-22,united kingdom,retail sales mom,retail sales mom,GBRRetailSalesMoM,0.20,-0.70,0.50,2024-12-20,0.90,-0.30
1048,2025-01-21,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.50,2.60,2.70,2025-01-15,-0.10,-0.20
1055,2025-01-22,united kingdom,monthly gdp mom,gdp mom,GBRMGM,0.00,-0.10,0.10,2025-01-16,0.10,-0.10


In [27]:
candles_df.head(2)

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,gain,pair
0,2024-08-27,81142,23.502,23.59858,23.45468,23.51225,23.4535,23.58961,23.43450,23.4795,23.5505,23.60765,23.46100,23.54500,0.043594,GBP_ZAR
1,2024-08-28,82376,23.516,23.53884,23.29209,23.40561,23.4590,23.52700,23.27637,23.3695,23.5730,23.58150,23.30781,23.44172,-0.471639,GBP_ZAR


In [28]:
candles_an = candles_df[['time', 'pair', 'gain']].copy()

In [29]:
candles_an.head(2)

,time,pair,gain
0,2024-08-27,GBP_ZAR,0.043594
1,2024-08-28,GBP_ZAR,-0.471639


In [30]:
merged = pd.merge(left=candles_an, right=calendar_data_df_uk, left_on='time', right_on='date')

In [32]:
merged.category.unique()

array(['unemployment rate', 'monthly gdp mom', 'inflation rate',
       'interest rate', 'retail sales mom', 'gdp growth rate',
       'gdp annual growth rate'], dtype=object)

In [34]:
merged[merged.category=="inflation rate"].head(2)

,time,pair,gain,date,country,category,event,symbol,actual,previous,forecast,orig_date,delta_prev,delta_fc
2,2024-09-18,GBP_ZAR,0.222801,2024-09-18,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,2.2,2.2,2.1,2024-09-18,0.0,0.1
7,2024-10-16,GBP_ZAR,0.484350,2024-10-16,united kingdom,inflation rate,inflation rate yoy,UKRPCJYR,1.7,2.2,2.0,2024-10-16,-0.5,-0.3


In [39]:
c = 'balance of trade'
df_an = merged[merged.category==c]
print(df_an[df_an.delta_prev >= 0].gain.sum())
print(df_an[df_an.delta_prev < 0].gain.sum())

0.0
0.0


In [42]:
import plotly.express as px

In [43]:
cat = 'inflation rate'
df_cat = merged[merged.category==cat]
for p in pairs:
    df_plot = df_cat[df_cat.pair==p]
    print(p)
    fig = px.scatter(df_plot, x="gain", y="delta_prev", trendline="ols")
    fig.show()

GBP_ZAR


GBP_HKD


GBP_CHF


GBP_AUD


GBP_CAD


...

GBP_USD


GBP_PLN


EUR_GBP


GBP_NZD


GBP_JPY
